In [9]:
import pandas as pd
import os
from sklearn import preprocessing  # pip install sklearn ... if you don't have it!
from collections import deque
import random
import numpy as np

# Longitud de la secuencia precedente para recolectar para la RNN
SEQ_LEN = 60

# Cuánto tiempo en el futuro estamos tratando de predecir
FUTURE_PERIOD_PREDICT = 8

# El ratio que queremos predecir
RATIO_TO_PREDICT = "BTC-USD"

# Función para clasificar si el valor futuro es mayor que el valor actual
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

def preprocess_df(df):
    df = df.drop(["future"], axis=1)

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic. Those nasty NaNs love to creep in.
    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.
    
    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array! ..import numpy as np

# Uso de una cadena sin procesar para evitar errores de escape de unicode
file_path = r"C:\Users\CRISTIAN CHAVEZ\Proyecto IA\cripto\crypto_data\BTC-USD.csv"

# Leer el archivo inicial
df = pd.read_csv(file_path, names=['time', 'low', 'high', 'open', 'close', 'volume'])

# DataFrame principal vacío para almacenar todos los datos
main_df = pd.DataFrame()

# Las 4 criptomonedas que queremos considerar
ratios = ["BCH-USD", "BTC-USD", "ETH-USD", "LTC-USD"]

# Ruta base donde se encuentran los archivos de datos
base_path = r"C:\Users\CRISTIAN CHAVEZ\Proyecto IA\cripto\crypto_data"

# Iterar sobre cada ratio
for ratio in ratios:
    print(ratio)
    
    # Obtener la ruta completa al archivo
    dataset = os.path.join(base_path, f"{ratio}.csv")
    
    # Leer el archivo específico
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    
    # Renombrar columnas de 'close' y 'volume' para incluir el ticker
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    
    # Establecer 'time' como índice para poder unirlos por este tiempo compartido
    df.set_index("time", inplace=True)
    
    # Ignorar otras columnas además de 'close' y 'volume'
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    # Si el DataFrame principal está vacío, simplemente asignar el DataFrame actual
    if len(main_df) == 0:
        main_df = df
    else:
        # De lo contrario, unir estos datos con el DataFrame principal
        main_df = main_df.join(df)

# Crear una nueva columna 'future' que es el valor de cierre futuro desplazado por 'FUTURE_PERIOD_PREDICT'
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

# Crear una columna 'target' que clasifica si el valor futuro es mayor que el valor actual
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

main_df.dropna(inplace=True)

# Imprimir las primeras filas del DataFrame resultante
#print(main_df.head())

times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times
print(last_5pct)
validation_main_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5pct)]  # now the main_df is all the data up to the last 5%train_x, train_y = preprocess_df(main_df) 

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)
print(f"train data: {len(train_x)} validation: {len(validation_x)}")


BCH-USD
BTC-USD
ETH-USD
LTC-USD
1534892940
train data: 79774 validation: 4042


In [10]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 79774 validation: 4042
Dont buys: 39887, buys: 39887
VALIDATION Dont buys: 2021, buys: 2021
